In [42]:
import pandas as pd 
import numpy as np

In [43]:
# links_small = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/links.csv')
# movies_small = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/movies.csv')
ratings_small = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/ratings.csv')
# tags_small = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest-small/tags.csv')

In [44]:
# links_full = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/links.csv')
# movies_full = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/movies.csv')
ratings_full = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/ratings.csv')
# tags_full = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/tags.csv')

In [45]:
gen_score = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/genome-scores.csv')
gen_tags = pd.read_csv('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/genome-tags.csv')

## Split data for Small file

In [46]:
# shuffle user
small_user_ct = ratings_small.groupby(by = "userId").count().shape[0]
np.random.seed(101)
ind = [i for i in range(small_user_ct)]
np.random.shuffle(ind)

In [47]:
# count number of records for each shuffled user 
ratings_small_gb = ratings_small.groupby(by = "userId").count().iloc[ind].reset_index()
ratings_small_gb

,userId,movieId,rating,timestamp
0,571,112,112,112
1,407,22,22,22
2,325,270,270,270
3,101,61,61,61
4,348,55,55,55
...,...,...,...,...
605,76,119,119,119
606,600,763,763,763
607,576,20,20,20
608,338,39,39,39


In [48]:
# get the cumulative count
ratings_small_gb["cumsum"] = ratings_small_gb["movieId"].cumsum()
ratings_small_gb

,userId,movieId,rating,timestamp,cumsum
0,571,112,112,112,112
1,407,22,22,22,134
2,325,270,270,270,404
3,101,61,61,61,465
4,348,55,55,55,520
...,...,...,...,...,...
605,76,119,119,119,99883
606,600,763,763,763,100646
607,576,20,20,20,100666
608,338,39,39,39,100705


In [49]:
# split the data to get the user id based the number of records
# train: 20%, test: 40%, validation: 40%
num_records_small = ratings_small.shape[0]

small_train_id = ratings_small_gb[ratings_small_gb["cumsum"] < num_records_small*0.2]

small_val_id = ratings_small_gb[(ratings_small_gb["cumsum"] >= num_records_small*0.2) & (ratings_small_gb["cumsum"] < num_records_small*0.6)]

small_test_id = ratings_small_gb[ratings_small_gb["cumsum"] >= num_records_small*0.6]


In [50]:
# find the ratings data corresponding to the user id for each splitted data
small_train = ratings_small.loc[ratings_small["userId"].isin(small_train_id["userId"])]
small_val = ratings_small.loc[ratings_small["userId"].isin(small_val_id["userId"])]
small_test = ratings_small.loc[ratings_small["userId"].isin(small_test_id["userId"])]

In [51]:
# generate the median timestamp value for each user in test&validation
small_val_grouped = small_val.groupby('userId')
small_val["median_time"] = small_val_grouped[["timestamp"]].transform(lambda x: x.median())

small_test_grouped = small_test.groupby('userId')
small_test["median_time"] = small_test_grouped[["timestamp"]].transform(lambda x: x.median())

/state/partition1/job-19640913/ipykernel_1431866/2490363789.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_val["median_time"] = small_val_grouped[["timestamp"]].transform(lambda x: x.median())
/state/partition1/job-19640913/ipykernel_1431866/2490363789.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  small_test["median_time"] = small_test_grouped[["timestamp"]].transform(lambda x: x.median())


In [52]:
# concat the first half(history) of the records of users in test&validation to the training set based on the median value of timestamp

small_train = pd.concat([small_train, small_val[small_val["timestamp"] < small_val["median_time"]].iloc[:,:4]]).reset_index(drop = True)
small_val = small_val[small_val["timestamp"] >= small_val["median_time"]].reset_index(drop = True)

small_train = pd.concat([small_train, small_test[small_test["timestamp"] < small_test["median_time"]].iloc[:,:4]]).reset_index(drop = True)
small_test = small_test[small_test["timestamp"] >= small_test["median_time"]].reset_index(drop = True)

## Split data for Full File

In [53]:
# shuffle user
full_user_ct = ratings_full.groupby(by = "userId").count().shape[0]
np.random.seed(101)
ind = [i for i in range(full_user_ct)]
np.random.shuffle(ind)

In [54]:
ratings_full.groupby(by = "userId").count()

,movieId,rating,timestamp
userId,,,
1,16,16,16
2,15,15,15
3,11,11,11
4,736,736,736
5,72,72,72
...,...,...,...
283224,329,329,329
283225,20,20,20
283226,11,11,11


In [55]:
# count number of records for each shuffled user 
ratings_full_gb = ratings_full.groupby(by = "userId").count().iloc[ind].reset_index()
ratings_full_gb

,userId,movieId,rating,timestamp
0,36672,28,28,28
1,118186,16,16,16
2,4364,43,43,43
3,10935,35,35,35
4,20885,31,31,31
...,...,...,...,...
283223,55294,17,17,17
283224,136768,170,170,170
283225,204615,14,14,14
283226,214540,11,11,11


In [56]:
# get the cumulative count
ratings_full_gb["cumsum"] = ratings_full_gb["movieId"].cumsum()
ratings_full_gb

,userId,movieId,rating,timestamp,cumsum
0,36672,28,28,28,28
1,118186,16,16,16,44
2,4364,43,43,43,87
3,10935,35,35,35,122
4,20885,31,31,31,153
...,...,...,...,...,...
283223,55294,17,17,17,27753180
283224,136768,170,170,170,27753350
283225,204615,14,14,14,27753364
283226,214540,11,11,11,27753375


In [57]:
# split the data to get the user id based the number of records
# train: 20%, test: 40%, validation: 40%
num_records_full = ratings_full.shape[0]

full_train_id = ratings_full_gb[ratings_full_gb["cumsum"] < num_records_full*0.2]

full_val_id = ratings_full_gb[(ratings_full_gb["cumsum"] >= num_records_full*0.2) & (ratings_full_gb["cumsum"] < num_records_full*0.6)]

full_test_id = ratings_full_gb[ratings_full_gb["cumsum"] >= num_records_full*0.6]


In [58]:
num_records_full*0.6

16652066.399999999

In [59]:
# find the ratings data corresponding to the user id for each splitted data
full_train = ratings_full.loc[ratings_full["userId"].isin(full_train_id["userId"])]
full_val = ratings_full.loc[ratings_full["userId"].isin(full_val_id["userId"])]
full_test = ratings_full.loc[ratings_full["userId"].isin(full_test_id["userId"])]

In [60]:
# generate the median timestamp value for each user in test&validation
full_val_grouped = full_val.groupby('userId')
full_val["median_time"] = full_val_grouped[["timestamp"]].transform(lambda x: x.median())

full_test_grouped = full_test.groupby('userId')
full_test["median_time"] = full_test_grouped[["timestamp"]].transform(lambda x: x.median())

/state/partition1/job-19640913/ipykernel_1431866/846529233.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_val["median_time"] = full_val_grouped[["timestamp"]].transform(lambda x: x.median())
/state/partition1/job-19640913/ipykernel_1431866/846529233.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  full_test["median_time"] = full_test_grouped[["timestamp"]].transform(lambda x: x.median())


In [61]:
# concat the first half(history) of the records of users in test&validation to the training set based on the median value of timestamp

full_train = pd.concat([full_train, full_val[full_val["timestamp"] < full_val["median_time"]].iloc[:,:4]]).reset_index(drop = True)
full_val = full_val[full_val["timestamp"] >= full_val["median_time"]].reset_index(drop = True)

full_train = pd.concat([full_train, full_test[full_test["timestamp"] < full_test["median_time"]].iloc[:,:4]]).reset_index(drop = True)
full_test = full_test[full_test["timestamp"] >= full_test["median_time"]].reset_index(drop = True)

In [63]:
full_train.to_csv("Data/ratings_full_train.csv", index = False, header = False)
full_val.to_csv("Data/ratings_full_val.csv", index = False, header = False)
full_test.to_csv("Data/ratings_full_test.csv", index = False, header = False)

In [64]:
small_train.to_csv("Data/ratings_small_train.csv", index = False, header = False)
small_val.to_csv("Data/ratings_small_val.csv", index = False, header = False)
small_test.to_csv("Data/ratings_small_test.csv", index = False, header = False)

In [ ]:
small_train

In [ ]:
aaa = small_train.groupby("movieId").agg({"rating": ["mean"]})
aaa.columns = aaa.columns.droplevel()
aaa = aaa.sort_values(by = "mean", ascending = False)

In [61]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
sparkContext=spark.sparkContext

ModuleNotFoundError: No module named 'pyspark'

In [58]:
predictionAndLabels = sparkContext.parallelize([
    ([1, 6, 2, 7, 8, 3, 9, 10, 4, 5], [1, 2, 3, 4, 5]),
    ([4, 1, 5, 6, 2, 7, 3, 8, 9, 10], [1, 2, 3]),
    ([1, 2, 3, 4, 5], [])])

NameError: name 'sc' is not defined

In [ ]:
# scp -r jz5246@peel.hpc.nyu.edu:/home/jz5246/ds1004/Project/final-project-group_101/Data/ratings_full_val.csv /Users/fengying/Desktop




In [ ]:
# scp -r jz5246@greene.hpc.nyu.edu:/home/jz5246/ds1004/Project/final-project-group_101/Data /Users/fengying/Desktop
# scp -r jz5246@gw.hpc.nyu.edu:/home/jz5246/ds1004/Project/final-project-group_101/Data /Users/fengying/Desktop
# scp -r jz5246@peel.hpc.nyu.edu:/home/jz5246/ds1004/Project/final-project-group_101/Data /Users/fengying/Desktop